In [1]:
from pyspark.sql.session import SparkSession
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.3.0"
]
spark = (SparkSession.builder.appName("MyDelta_App")
    .config("spark.jars.packages", ",".join(maven_coords))  
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", "512m")
    .config("spark.driver.memory", "1g")
    .config('spark.ui.port', '4040')
    .getOrCreate()
    )
spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8fb219ff-8ea2-4db5-ac6d-339038b5396d;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.2.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 228ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.spark#spark-avro_2.12;3.2.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.

24/07/16 23:41:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
from pyspark.sql.types import *
from datetime import datetime

schema = StructType(
          [StructField("name", StringType(), False),
           StructField("dob", DateType(), False)])

df = spark.createDataFrame([["ash", datetime.strptime("2020-01-01", "%Y-%m-%d")]], schema = schema)

df.write.mode("overwrite").format("delta").saveAsTable("delta_table")

In [ ]:
spark.sql("select * from delta_table").show()

In [5]:
import delta

In [10]:
from delta.tables import *

schema = StructType(
          [StructField("new_name", StringType(), False),
           StructField("new_dob", DateType(), False)
          ]
        )
new_df = spark.createDataFrame([["ash", datetime.strptime("2025-01-01", "%Y-%m-%d")]], schema = schema)

print("Original dataframe...")
spark.sql("select * from delta_table").show()

delta_table_ref = DeltaTable.forName(spark, "delta_table")
delta_table_ref.merge(new_df, "name=new_name") \
  .whenMatchedUpdate(set =
    {
      "dob": "new_dob"
    }
    ) \
  .execute()

print("after merge...")
spark.sql("select * from delta_table").show()

Original dataframe...
+----+----------+
|name|       dob|
+----+----------+
| ash|2025-01-01|
+----+----------+

after merge...
+----+----------+
|name|       dob|
+----+----------+
| ash|2025-01-01|
+----+----------+



In [ ]:
schema = StructType(
          [StructField("name", StringType(), False),
           StructField("dob", DateType(), False)
          ]        )
df = spark.createDataFrame([["ash", datetime.strptime("2010-01-01", "%Y-%m-%d")]], schema = schema)

new_df = spark.createDataFrame([["ash", datetime.strptime("2026-01-01", "%Y-%m-%d")],
                                ["mat", datetime.strptime("9926-01-01", "%Y-%m-%d")]], schema = schema)

df.write.mode("overwrite").format("delta").saveAsTable("my_delta")
new_df.write.mode("overwrite").format("delta").saveAsTable("updates_to_delta")

print("Original dataframe...")
spark.sql("select * from my_delta").show()

spark.sql("""Merge into my_delta
              using updates_to_delta
              on my_delta.name = updates_to_delta.name
              when matched then
                update set
                  dob = updates_to_delta.dob
              when not matched then
                insert (name, dob)  values (updates_to_delta.name, updates_to_delta.dob)

            """)
print("after merging using sql api ...")
spark.sql("select * from my_delta").show()